# Intro to NLP

## Course work

In [1]:
!pip install pymorphy2[fast] annoy stop_words transformers

     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 646 kB 32.1 MB/s 
     |████████████████████████████████| 3.1 MB 39.9 MB/s 
     |████████████████████████████████| 895 kB 48.4 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.4 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 8.2 MB 31.9 MB/s 
     |████████████████████████████████| 371 kB 45.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391658 sha256=06fa92db742929978b2b2ff230ca95c86af08164ed7a6b695bc675c78cf5e652
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=a18c086b6c86cd800cbc55c7da7107fae19f683f724fc0011f70fbcf051928d5
  Stored in directory: /root/.cache/pip/wheels/fb/86

In [2]:
import string
import nltk
import annoy
import numpy as np
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
import pickle
import os
import pandas as pd

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/data/gb_NLP_cv/'

Mounted at /content/drive


In [4]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

def preprocess_txt(line):
  spls = "".join(i for i in line.strip() if i not in exclude).split()
  spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
  spls = [i for i in spls if i not in sw and i != ""]
  return ' '.join(spls)

## Fetching database

In [5]:
import sqlite3 as sql


cur = sql.connect(f'{path}jokes.db').execute('SELECT theme,text FROM joke')
rows = cur.fetchall()
cur.close()

jokes = {idx: {'theme': itm[0], 'text': itm[1]} for idx,itm in enumerate(rows)}

2021-11-02 16:15:38,984 INFO sqlalchemy.engine.Engine SELECT theme,text FROM joke
2021-11-02 16:15:38,985 INFO sqlalchemy.engine.Engine [raw sql] ()


In [6]:
print(f'Theme: {jokes[0]["theme"]}\n')
print(f'Text: {jokes[0]["text"]}')

Theme: pro-sudey

Text: На суде в Стамбуле обвиняемый сказал:
- На свои жертвы я нападал всегда днем. Ночью я бы побоялся
ходить с награбленными деньгами...




## Tokenizing the text

In [7]:
df = pd.DataFrame.from_dict(jokes, columns=['theme', 'text'], orient='index')
df['text_token'] = df['text'].apply(lambda x: preprocess_txt(x))
df.head()

,theme,text,text_token
0,pro-sudey,На суде в Стамбуле обвиняемый сказал:\r\n- На...,суд стамбул обвиняемый свой жертва нападать н...
1,pro-sudey,"- Вы продолжаете утверждать, что обвиняемый н...",продолжать утверждать обвиняемый назвать дура...
2,pro-sudey,"На суде.\r\n- Итак, когда дело дошло до столкн...",суд итак дело дойти столкновение противник рук...
3,pro-sudey,Старую леди сбил автомобиль. На суде ее спраши...,старый леди сбить автомобиль суд спрашивать де...
4,pro-sudey,"Судья говорит:\r\n- Согласно вашей жалобе, об...",судья говорить согласно вашей жалоба обвиняем...


In [8]:
print(df.loc[0, 'text'])
print(df.loc[0, 'text_token'])

На суде в Стамбуле обвиняемый сказал:
- На свои жертвы я нападал всегда днем. Ночью я бы побоялся
ходить с награбленными деньгами...


суд стамбул обвиняемый свой жертва нападать ночью побояться ходить награбить деньга


## Target feature encoding

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['theme'] = le.fit_transform(df['theme'])
df.head()

,theme,text,text_token
0,34,На суде в Стамбуле обвиняемый сказал:\r\n- На...,суд стамбул обвиняемый свой жертва нападать н...
1,34,"- Вы продолжаете утверждать, что обвиняемый н...",продолжать утверждать обвиняемый назвать дура...
2,34,"На суде.\r\n- Итак, когда дело дошло до столкн...",суд итак дело дойти столкновение противник рук...
3,34,Старую леди сбил автомобиль. На суде ее спраши...,старый леди сбить автомобиль суд спрашивать де...
4,34,"Судья говорит:\r\n- Согласно вашей жалобе, об...",судья говорить согласно вашей жалоба обвиняем...


In [10]:
with open(os.path.join(path, 'label_encoder.pkl'), 'wb') as file:
  pickle.dump(le, file)

## Train SVC model

In [11]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


X_train, X_test, y_train, y_test = train_test_split(
  df['text_token'], df['theme'],
  test_size=0.2,
  stratify=df['theme'],
  random_state=100
)

tfidf = TfidfVectorizer(
  ngram_range=(1, 2),
  max_features=10000
)

X_train_idf = tfidf.fit_transform(X_train)
X_test_idf = tfidf.transform(X_test)

svc = LinearSVC(
    random_state=100,
    max_iter=1000,
    loss='squared_hinge',
    dual=False,
)
svc.fit(X_train_idf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=100, tol=0.0001,
          verbose=0)

In [12]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


def calc_score(model, X, y):
  cv = cross_val_score(
      model,
      X, y,
      cv=5,
      scoring='f1_weighted'
  )
  print(f'F1_weighted cv: {np.mean(cv)}')
  print(f'\nClassification report\n{classification_report(y, model.predict(X))}')

calc_score(svc, X_test_idf, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


F1_weighted cv: 0.5459447624943581

Classification report
              precision    recall  f1-score   support

           0       0.67      0.82      0.74      7106
           1       0.22      0.02      0.04       104
           2       0.33      0.10      0.15        94
           3       0.50      0.09      0.15        67
           4       0.41      0.21      0.28       398
           5       0.30      0.09      0.14       288
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         4
           8       0.28      0.05      0.09       430
           9       0.95      0.83      0.89       230
          10       0.63      0.47      0.54       454
          11       0.78      0.91      0.84        23
          12       0.97      0.66      0.78        47
          13       0.83      0.89      0.86        91
          14       0.89      0.85      0.87       419
          15       0.00      0.00      0.00         3
          16       0.77

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Train FastText model

In [18]:
class MyIter:
  def __iter__(self):
    for index, row in df.iterrows():
      yield row['text_token']


modelFT = FastText(sentences=MyIter(), size=30, min_count=1, window=5, workers=8)
modelFT.save(os.path.join(path, 'model_FT.ft'))

In [25]:
ft_index = annoy.AnnoyIndex(30 ,'angular')

idfs = {v[0]: v[1] for v in zip(tfidf.vocabulary_, tfidf.idf_)}
midf = np.mean(tfidf.idf_)

index_map = {}
counter = 0

for index, row in df.iterrows():
    n_ft = 0
    index_map[counter] = (df.loc[index, "theme"], df.loc[index, "text"], df.loc[index, "text_token"])
    vector_ft = np.zeros(30)
    for word in df.loc[index, "text_token"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index.add_item(counter, vector_ft)
    counter += 1

ft_index.build(10)
ft_index.save(os.path.join(path, 'ft_index.ann'))

with open(os.path.join(path, 'index_map.pkl'), 'wb') as file:
  pickle.dump(index_map, file)

with open(os.path.join(path, 'tfidf.pkl'), 'wb') as file:
  pickle.dump(tfidf, file)

In [23]:
def embed_txt(txt, idfs, midf):
  n_ft = 0
  vector_ft = np.zeros(30)
  for word in txt:
    if word in modelFT.wv:
      vector_ft += modelFT.wv[word] * idfs.get(word, midf)
      n_ft += idfs.get(word, midf)
  return vector_ft / n_ft

def get_predictions(text):
  text = preprocess_txt(text)
  text_ = tfidf.transform([text])
  theme = svc.predict(text_)

  vect_ft = embed_txt(text, idfs, midf)
  ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 10, include_distances=True)
  joke = None
  for item, dist in zip(ft_index_val, distances):
    if dist <= 0.25:
      joke = index_map[item][1]
    break
  if joke:
    return joke
  
  joke = df.loc[df["theme"] == theme[0]].sample()['text'].values[0]
  return joke

In [24]:
for word in ['путин', 'деньги', 'чукча', 'штирлиц', 'мент', 'про рабиновича', 'женщина', 'алкоголик', 'судья', 'чукча', 'про поручика']:
  print(f'word={word}')
  print(get_predictions(word))

путин
Из сводок новостей: Борис Николаевич Ельцин неожиданно прервал свой отпуск и вылетел в Москву. Зайдя в спальню, он обнаружил там: Наину Иосифовну и некоего мужчину (в шкафу) , для которых все это было также неожиданно. Мужчиной оказался премьер правительства Российской Федерации Владимир Владимирович Путин, с которым Борис Николаевич Ельцин срочно провел совещание.

деньги
Мы всю жизнь бегаем за деньгами, а деньги от нас.



чукча
Чукча подходит к проститутке на Тверской:
- Скока за ночь, однака?
Она смерила его взглядом:
- С тебя 150 баксов, лох.
Тут подскакивает чукотская братва и запихивает ее в багажник. Прошло полгода... ее высаживают на том же месте, одичавшую, вонючую, в унтах и чукотских тряпках: в школе она училась плохо, откуда ей было знать, что ночь полярная.

штирлиц
Штирлиц ехал по шоссе. Вдруг он увидел голосующего Мюллера.
- Не могу я подвозить этого палача, погубившего миллионы советских
людей! - подумал Штирлиц и проехал мимо.
Через минуту он опять увид